In [35]:
import strym
from strym import strymread
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pymysql
import re

In [41]:
file= "2021-04-06-15-13-00_2T3Y1RFV8KC014025_CAN_Messages.csv"
r =strymread(csvfile=file)

[2022_02_06_23_13_51] (root) INFO: Vehicle model infered is toyota-rav4-2019


In [11]:
#regex match
vehicle_id_pattern = r"\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}_([A-Z0-9]*)_CAN_Messages.csv"
vehicle_id = re.findall(vehicle_id_pattern,file)[0]
vehicle_id

'2T3Y1RFV8KC014025'

In [ ]:
lead_distance = r.lead_distance()['Message']
lead_distance_time = pd.to_datetime(r.lead_distance()['Time'],unit='s')

In [ ]:
relative_vel = r.relative_leadervel()['Message']
relative_vel_time = pd.to_datetime(r.relative_leadervel()['Time'],unit='s')

In [48]:
def create_feature_dataframe(df_raw,feature_name):
    vehicle_id_pattern = r"\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}_([A-Z0-9]*)_CAN_Messages.csv"
    vehicle_id = re.findall(vehicle_id_pattern,file)[0]
    feature = df_raw['Message']
    time =  pd.to_datetime(df_raw['Time'],unit='s')
    df_feature = pd.DataFrame()
    df_feature[feature_name]=feature
    df_feature= df_feature.reset_index()
    df_feature.rename(columns={'Clock':'time'},inplace=True)
    df_feature['vehicle_id'] =vehicle_id
    return df_feature
def mysql_batch_update(dataframe,table_name):
    from sqlalchemy import create_engine
    engine = create_engine("mysql+pymysql://{}:{}@{}/{}".format('capstone', 'capstone123!!!', 'sh-cynosdbmysql-grp-75qwodo8.sql.tencentcdb.com:29563', 'circle_database_prototype'))
    con = engine.connect()
    return dataframe.to_sql(name=table_name, con=con, if_exists='append', index=False)

In [49]:
mysql_batch_update(create_feature_dataframe(r.accelx(),'acceleration'),'raw_vehicle_acceleration')

In [5]:
speed = r.speed()['Message']
speed_time = pd.to_datetime(r.speed()['Time'],unit='s')

In [7]:
# db = pymysql.connect(host='sh-cynosdbmysql-grp-75qwodo8.sql.tencentcdb.com',port =29563,user='capstone',password='capstone123!!!')
# cursor = db.cursor()
# sql = "CREATE DATABASE IF NOT EXISTS circle_database_prototype" 
# cursor.execute(sql)

In [36]:
# ##create the features table
# db = pymysql.connect(host='sh-cynosdbmysql-grp-75qwodo8.sql.tencentcdb.com',port =29563,user='capstone',password='capstone123!!!',db='circle_database_prototype')
# cursor = db.cursor()
# # Create table vehicle_speed
# cursor.execute("DROP TABLE IF EXISTS raw_vehicle_speed")
# sql = """CREATE TABLE `raw_vehicle_speed` (
#   `id` int(11) NOT NULL AUTO_INCREMENT,
#   `vehicle_id` char(128) NOT NULL,
#   `speed` float(10,2) DEFAULT NULL,
#   `time` timestamp(3) NOT NULL DEFAULT CURRENT_TIMESTAMP(3) ON UPDATE CURRENT_TIMESTAMP(3),
#   PRIMARY KEY (`id`)
# ) ENGINE=InnoDB AUTO_INCREMENT=0 DEFAULT CHARSET=utf8;"""
# cursor.execute(sql)
# # Create table vehicle_relative_vel
# cursor.execute("DROP TABLE IF EXISTS raw_vehicle_relative_vel")
# sql = """CREATE TABLE `raw_vehicle_relative_vel` (
#   `id` int(11) NOT NULL AUTO_INCREMENT,
#   `vehicle_id` char(128) NOT NULL,
#   `relative_vel` float(10,2) DEFAULT NULL,
#   `time` timestamp(3) NOT NULL DEFAULT CURRENT_TIMESTAMP(3) ON UPDATE CURRENT_TIMESTAMP(3),
#   PRIMARY KEY (`id`)
# ) ENGINE=InnoDB AUTO_INCREMENT=0 DEFAULT CHARSET=utf8;"""
# cursor.execute(sql)
# # Create table lead_distance
# cursor.execute("DROP TABLE IF EXISTS cleaned_vehicle_lead_distance")
# sql = """CREATE TABLE `cleaned_vehicle_lead_distance` (
#   `id` int(11) NOT NULL AUTO_INCREMENT,
#   `vehicle_id` char(128) NOT NULL,
#   `lead_distance` float(10,2) DEFAULT NULL,
#   `time` timestamp(3) NOT NULL DEFAULT CURRENT_TIMESTAMP(3) ON UPDATE CURRENT_TIMESTAMP(3),
#   PRIMARY KEY (`id`)
# ) ENGINE=InnoDB AUTO_INCREMENT=0 DEFAULT CHARSET=utf8;"""
# cursor.execute(sql)
# print("Created table Successfull.")
# Create table vehicle_speed
# cursor.execute("DROP TABLE IF EXISTS raw_vehicle_acceleration")
# sql = """CREATE TABLE `raw_vehicle_acceleration` (
#   `id` int(11) NOT NULL AUTO_INCREMENT,
#   `vehicle_id` char(128) NOT NULL,
#   `acceleration` float(10,2) DEFAULT NULL,
#   `time` timestamp(3) NOT NULL DEFAULT CURRENT_TIMESTAMP(3) ON UPDATE CURRENT_TIMESTAMP(3),
#   PRIMARY KEY (`id`)
# ) ENGINE=InnoDB AUTO_INCREMENT=0 DEFAULT CHARSET=utf8;"""
# cursor.execute(sql)

# # disconnect from server
# db.close()

In [10]:
df_speed = pd.DataFrame()
df_speed['speed']=speed
df_speed= df_speed.reset_index()
df_speed.rename(columns={'Clock':'time'},inplace=True)
df_speed['vehicle_id'] =vehicle_id
df_speed.head(5)

,time,speed,vehicle_id
0,2021-04-06 15:13:01.167064064,0.0,2T3Y1RFV8KC014025
1,2021-04-06 15:13:01.169282048,0.0,2T3Y1RFV8KC014025
2,2021-04-06 15:13:01.173085952,0.0,2T3Y1RFV8KC014025
3,2021-04-06 15:13:01.174201856,0.0,2T3Y1RFV8KC014025
4,2021-04-06 15:13:01.180964864,0.0,2T3Y1RFV8KC014025


In [16]:
from sqlalchemy import create_engine
engine = create_engine("mysql+pymysql://{}:{}@{}/{}".format('capstone', 'capstone123!!!', 'sh-cynosdbmysql-grp-75qwodo8.sql.tencentcdb.com:29563', 'circle_database_prototype'))
con = engine.connect()
df_speed.to_sql(name='raw_vehicle_speed', con=con, if_exists='append', index=False)

In [17]:
df_lead_distance = pd.DataFrame()
df_lead_distance['lead_distance']=lead_distance
df_lead_distance= df_lead_distance.reset_index()
df_lead_distance.rename(columns={'Clock':'time'},inplace=True)
df_lead_distance['vehicle_id'] =vehicle_id

In [8]:
from sqlalchemy import create_engine
engine = create_engine("mysql+pymysql://{}:{}@{}/{}".format('capstone', 'capstone123!!!', 'sh-cynosdbmysql-grp-75qwodo8.sql.tencentcdb.com:29563', 'circle_database_prototype'))
con = engine.connect()
df_lead_distance.to_sql(name='raw_vehicle_lead_distance', con=con, if_exists='append', index=False)

In [19]:
df_relative_vel = pd.DataFrame()
df_relative_vel['relative_vel']=relative_vel
df_relative_vel= df_relative_vel.reset_index()
df_relative_vel.rename(columns={'Clock':'time'},inplace=True)
df_relative_vel['vehicle_id'] =vehicle_id
df_relative_vel

,time,relative_vel,vehicle_id
0,2021-04-06 15:13:01.168414976,0.000,2T3Y1RFV8KC014025
1,2021-04-06 15:13:01.168414976,0.000,2T3Y1RFV8KC014025
2,2021-04-06 15:13:01.168634880,0.000,2T3Y1RFV8KC014025
3,2021-04-06 15:13:01.168634880,0.000,2T3Y1RFV8KC014025
4,2021-04-06 15:13:01.168634880,0.000,2T3Y1RFV8KC014025
...,...,...,...
107793,2021-04-06 15:23:39.803931904,0.000,2T3Y1RFV8KC014025
107794,2021-04-06 15:23:39.852005888,-1.225,2T3Y1RFV8KC014025
107795,2021-04-06 15:23:39.852215040,0.000,2T3Y1RFV8KC014025
107796,2021-04-06 15:23:39.853068032,0.000,2T3Y1RFV8KC014025


In [15]:
from sqlalchemy import create_engine
engine = create_engine("mysql+pymysql://{}:{}@{}/{}".format('capstone', 'capstone123!!!', 'sh-cynosdbmysql-grp-75qwodo8.sql.tencentcdb.com:29563', 'circle_database_prototype'))
con = engine.connect()
df_relative_vel.to_sql(name='raw_vehicle_relative_vel', con=con, if_exists='append', index=False)

In [32]:
#serach
db = pymysql.connect(host='sh-cynosdbmysql-grp-75qwodo8.sql.tencentcdb.com',port =29563,user='capstone',password='capstone123!!!',db='circle_database_prototype')
cursor = db.cursor()
sql = "SELECT lead_distance, time FROM raw_vehicle_lead_distance WHERE time > '2021-04-06 01:00:00' and time < '2021-04-06 23:00:00' and vehicle_id = '2T3Y1RFV8KC014025'"
cursor.execute(sql)
# Fetch all the rows in a list of lists.
results = cursor.fetchall()
db.close()


In [34]:
list(results)

TypeError: 'list' object is not callable

In [29]:
list(results)

TypeError: 'list' object is not callable

In [24]:
x

((252.0, datetime.datetime(2021, 4, 6, 15, 13, 1, 273000)),
 (252.0, datetime.datetime(2021, 4, 6, 15, 13, 1, 456000)),
 (252.0, datetime.datetime(2021, 4, 6, 15, 13, 1, 457000)),
 (252.0, datetime.datetime(2021, 4, 6, 15, 13, 1, 657000)),
 (252.0, datetime.datetime(2021, 4, 6, 15, 13, 1, 657000)),
 (252.0, datetime.datetime(2021, 4, 6, 15, 13, 1, 857000)),
 (252.0, datetime.datetime(2021, 4, 6, 15, 13, 2, 57000)),
 (252.0, datetime.datetime(2021, 4, 6, 15, 13, 2, 57000)),
 (252.0, datetime.datetime(2021, 4, 6, 15, 13, 2, 257000)),
 (252.0, datetime.datetime(2021, 4, 6, 15, 13, 2, 257000)),
 (252.0, datetime.datetime(2021, 4, 6, 15, 13, 2, 457000)),
 (252.0, datetime.datetime(2021, 4, 6, 15, 13, 2, 457000)),
 (252.0, datetime.datetime(2021, 4, 6, 15, 13, 2, 657000)),
 (252.0, datetime.datetime(2021, 4, 6, 15, 13, 2, 657000)),
 (252.0, datetime.datetime(2021, 4, 6, 15, 13, 2, 857000)),
 (252.0, datetime.datetime(2021, 4, 6, 15, 13, 3, 57000)),
 (252.0, datetime.datetime(2021, 4, 6, 15, 